# Recurrent Neural Networks

Works much better on **sequential** data such as financial series, text and audio.

#### Motivation
* Many supervised learning problems deal with ordered sequences (e.g. predicting stock prices)
    * `input`: ordered sequence of past series values
    * `output`: ordered sequence of futures series values
    
**Ressources:**
* [academic RNN text generator](http://www.cs.toronto.edu/~ilya/rnn.html)
* [twitter bots](http://tweet-generator-alex.herokuapp.com/)
* [NanoGenMo](https://github.com/NaNoGenMo/2016)
* [Robot Shakespear](https://github.com/genekogan/RobotShakespeare)

## 1. Ordered Sequences

* **Ordererd sequence**: a list of values ordered by index `(S1,S2,S3,...SP)`
    * index can be time stamp or other orders of sequence
    
* Many real ordered sequences are a product of some underlying process or processes
* But underlying process(es) can be a black box, with no model that explains the data
* Instead we broadly model ordered sequences **recursively**:
    * use past values in a sequence to predict future ones
    * we model future values of a sequence mathematically in terms of it's predecessors

* **seed**: initial value(s) of a recursive sequence
* **order**: number of elements future values are dependent on